In [0]:
#Instalations
%pip install databricks-vectorsearch
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Input Query  
question = dbutils.widgets.get('query')

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3894095008779751>, line 2
      1 # Input Query
----> 2 question = dbutils.widgets.get('query')

File /databricks/python_shell/dbruntime/WidgetHandlerImpl.py:77, in WidgetsHandlerImpl.get(self, name)
     37 def get(self, name):
     38     """ Returns the current value of a widget with the given name.
     39 
     40     :param name: Name of the argument to be accessed
   (...)
     75         ```
     76     """
---> 77     return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *args)
   1349 command = proto.CALL_COMMAND_NAME +\
   1350     self.command_header +\
   1351     args_command +\
   1352     proto.END_COMMAND_PART
   1354 answer = self.gateway_client.send_command(c

In [0]:
# Querying the Vector Search Index
import os
from databricks.vector_search.client import VectorSearchClient
workspace_url = os.environ.get("WORKSPACE_URL")
sp_client_id = os.environ.get("SP_CLIENT_ID")
sp_client_secret = os.environ.get("SP_CLIENT_SECRET")
vsc = VectorSearchClient(
    workspace_url=workspace_url,
    service_principal_client_id=sp_client_id,
    service_principal_client_secret=sp_client_secret
)

results = vsc.get_index("spectra_classification1","main.default.spectraclassification1_index").similarity_search(
  query_text=question,
  columns=["Chunks"],
  num_results=5)
rest = results.get('result', {}).get('data_array', [])
rest

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3894095008779740>, line 14
      6 sp_client_secret = os.environ.get("SP_CLIENT_SECRET")
      7 vsc = VectorSearchClient(
      8     workspace_url=workspace_url,
      9     service_principal_client_id=sp_client_id,
     10     service_principal_client_secret=sp_client_secret
     11 )
     13 results = vsc.get_index("spectra_classification1","main.default.spectraclassification1_index").similarity_search(
---> 14   query_text=question,
     15   columns=["Chunks"],
     16   num_results=5)
     17 rest = results.get('result', {}).get('data_array', [])
     18 rest

NameError: name 'question' is not defined

In [0]:
# For sql query generation
chain_config = {
    "llm_model_serving_endpoint_name": "databricks-meta-llama-3-1-405b-instruct",
    "vector_search_endpoint_name": "spectra_classification1",
    "vector_search_index": "main.default.spectraclassification1_index",
    "llm_prompt_template": """You are a SQL expert. Based on the classification information provided and the input query, generate a syntactically correct SQL query to run and return ONLY the generated query and nothing else.\n\n\
        The classification information contains details about the relevant subtopics, categories, taxonomy, attributes, entities, and relationships between tables. You must use this information to guide the construction of the SQL query.\n\n\
        Classification Chunks: {context}\n\n\
        Given this information, generate a SQL query that adheres to the following rules:\n\n\
        - **Identify the correct table for each attribute or metric**: Ensure that the SQL query correctly identifies the table from which to retrieve each attribute (e.g., budgets should be retrieved from the **Project** table, not the **Department** table).\n\
        - **Use the correct aggregation functions**: If the query involves summing, averaging, or counting, make sure the correct aggregation function is applied on the appropriate column (e.g., `SUM(Budget)` when summing project budgets).\n\
        - **Perform accurate JOIN operations**: Ensure that JOINs are used where necessary based on the relationships between tables (e.g., `Project` and `Department` using `Department_ID`). The JOIN should reflect the relationships outlined in the classification information. Consider all types of JOINS\n\
        - **Use filtering conditions properly**: When the query asks for specific filtering criteria (e.g., department name, project status, date ranges), make sure to use appropriate filtering conditions like `WHERE`, `AND`, or `OR`.\n\
        - **Handle date and time-related queries**: When dealing with date or timestamp columns, use the correct functions (e.g., `EXTRACT`, `DATE_PART`, `YEAR`) to filter or aggregate date-related data correctly.\n\
        - **Group results where necessary**: If the query requires grouping data (e.g., summing by department, counting by project), ensure that `GROUP BY` is used correctly along with aggregate functions.\n\
        - **Limit rows when necessary**: Ensure that the number of returned rows is restricted if needed, by using `LIMIT` or ensuring only necessary rows are retrieved.\n\
        - **Check for aliasing where appropriate**: Use aliases to improve query readability, especially when working with multiple tables or complex joins.\n\
        -**Read the labels correctly**: Make sure the labels are correct and match the classification information and there are some labels which are similar with slight differences like a underscore. Treat them as different columns.(e.g., department_id from T2 is different from deparatment from T1).\n\
        - Do not include any new-line characters or unnecessary symbols.\n\
        - Return only the generated SQL query without any additional explanations."""
}


In [0]:
# User input to SQL query conversion
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks
prompt = ChatPromptTemplate.from_messages(
    [  
        ("system", chain_config.get("llm_prompt_template")),  
        ("user", "{question}")  # User's input query
    ]
)

# Model for answering using Databricks endpoint
model = ChatDatabricks(
    endpoint=chain_config.get("llm_model_serving_endpoint_name"),
    extra_params={"temperature": 0.01, "max_tokens": 500}
)

# Generate the final SQL query based on the user's question and context
answer = prompt | model
response = answer.invoke({'question': question, 'context': rest})

# Extract the SQL query from the response
sql = response.content


In [0]:
%sql
USE spectra_tables

In [0]:
# Executing the query using Spark SQL
output_string = ""
result_df = spark.sql(sql)
result = result_df.collect()

if len(result) == 1 and len(result[0]) == 1:
    # Single row, single column result- (e.g., COUNT, SUM)
    query_result = result[0][0]
    output_string = str(query_result)
else: 
    for row in result:
    # Convert each row to a string format by iterating over all columns in the row
        row_string = ', '.join([str(col) for col in row])
        output_string += row_string + "\n"

output_string = output_string.strip()
print(output_string)

In [0]:
dbutils.notebook.exit(output_string)